# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
MAILS_FN = 'enron_mails_small.csv'

In [3]:
mails = sc.textFile(MAILS_FN, use_unicode=True).cache() 

In [4]:
def extractName(partitionId, row):
    if partitionId == 0:
        next(row)
    import csv
    reader = csv.reader(row)
    for row in reader:
        for i in row[2].split(';'):
            if i.split('@')[1] == 'enron.com' and row[1].split('@')[1] == 'enron.com':
                
                # extract the from_name
                for j in range(len(row[1].split('@')[0].split('.'))):
                    if len(row[1].split('@')[0].split('.')) == 1:
                        from_name = row[1].split('@')[0].split('.').capitalize()
                    elif len(row[1].split('@')[0].split('.')) == 2:
                        from_name = row[1].split('@')[0].split('.')[0].capitalize() +" "+  row[1].split('@')[0].split('.')[1].capitalize()
                    else:
                        from_name = row[1].split('@')[0].split('.')[0].capitalize() +" "+  row[1].split('@')[0].split('.')[1].capitalize() + row[1].split('@')[0].split('.')[2].capitalize()

                # extract the to_name        
                for k in range(len(i.split('@')[0].split('.'))):
                    if len(i.split('@')[0].split('.')) == 1:
                        to_name = i.split('@')[0].split('.')[0].capitalize()
                    elif len(i.split('@')[0].split('.')) == 2:
                        to_name = i.split('@')[0].split('.')[0].capitalize() +" "+ i.split('@')[0].split('.')[1].capitalize()
                    else:
                        to_name = i.split('@')[0].split('.')[0].capitalize() +" "+ i.split('@')[0].split('.')[1].capitalize()+" "+ i.split('@')[0].split('.')[2].capitalize()
              
                # sort each pair of from_name and to_name and give them a number 0 or 1
                if from_name < to_name:
                    yield ((from_name, to_name), 0)
                else:
                    yield ((to_name, from_name), 1)

In [5]:
result = mails.mapPartitionsWithIndex(extractName).groupByKey() \
    .filter(lambda x : (len(set(x[1]))==2)) \
    .sortByKey()\
    .map(lambda x: ('recipropcal', '%s : %s' % (x[0][0], x[0][1]))) \
    .collect()

# groupByKey: (sorted(from_name,to_name), list(0,1,....))
# filter: if x[1] has 0 and 1, it means this pair of names are recipropal
# sortByKey(): sort the result of pairs of names

print(len(result))
result

35


[('recipropcal', 'Brenda Whitehead : Elizabeth Sager'),
 ('recipropcal', 'Carol Clair : Debra Perlingiere'),
 ('recipropcal', 'Carol Clair : Mark Taylor'),
 ('recipropcal', 'Carol Clair : Richard Sanders'),
 ('recipropcal', 'Carol Clair : Sara Shackleton'),
 ('recipropcal', 'Carol Clair : Tana Jones'),
 ('recipropcal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('recipropcal', 'Drew Fossum : Susan Scott'),
 ('recipropcal', 'Elizabeth Sager : Janette Elbertson'),
 ('recipropcal', 'Elizabeth Sager : Mark Haedicke'),
 ('recipropcal', 'Elizabeth Sager : Mark Taylor'),
 ('recipropcal', 'Elizabeth Sager : Richard Sanders'),
 ('recipropcal', 'Eric Bass : Susan Scott'),
 ('recipropcal', 'Fletcher Sturm : Greg Whalley'),
 ('recipropcal', 'Fletcher Sturm : Sally Beck'),
 ('recipropcal', 'Gerald Nemec : Susan Scott'),
 ('recipropcal', 'Grant Masson : Vince Kaminski'),
 ('recipropcal', 'Greg Whalley : Richard Sanders'),
 ('recipropcal', 'Janette Elbertson : Mark Taylor'),
 ('recipropcal', 'Janette El

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

In [6]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [7]:
sat = sc.textFile(SAT_FN, use_unicode=True).cache()
list(enumerate(sat.first().split(','))) # the column names

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [8]:
def extractScore(partitionId, rows):
    if partitionId==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if fields[2]!='s':      # the null in data is 's'
            yield (fields[0], (int(fields[4]), int(fields[2])))

satScores = sat.mapPartitionsWithIndex(extractScore)
satScores.take(5)

[('02M047', (400, 16)),
 ('21K410', (437, 475)),
 ('30Q301', (440, 98)),
 ('17K382', (374, 59)),
 ('18K637', (381, 35))]

In [9]:
schools = sc.textFile(HSD_FN, use_unicode=True).cache()
list(enumerate(schools.first().split(',')))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

In [10]:
# Bus
def extractSchool_bus(partitionId, rows):
    if partitionId == 0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) == 58 and fields[10] != 'N/A': # there are N/A in the bus and subway columns
            for i in range(len(fields[10].split(', '))):
                yield(fields[0], fields[10].split(', ')[i])
    
school_bus = schools.mapPartitionsWithIndex(extractSchool_bus)
school_bus.take(10)

[('01M292', 'B39'),
 ('01M292', 'M14A'),
 ('01M292', 'M14D'),
 ('01M292', 'M15'),
 ('01M292', 'M15-SBS'),
 ('01M292', 'M21'),
 ('01M292', 'M22'),
 ('01M292', 'M9'),
 ('01M448', 'M14A'),
 ('01M448', 'M14D')]

In [14]:
school_bus.join(satScores).values() \
    .mapValues(lambda x: (x[0]*x[1], x[1])) \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: int(x[0]/x[1])) \
    .sortBy(lambda x: x[1], ascending=False) \
    .take(100)
# print the bus lines with top 10 SAT scores

[('S1115', 612),
 ('M79', 594),
 ('Q42', 582),
 ('M22', 574),
 ('Bx3', 571),
 ('B52', 560),
 ('B63', 557),
 ('B69', 548),
 ('B54', 543),
 ('B25', 541),
 ('M20', 540),
 ('M9', 539),
 ('B65', 538),
 ('M86', 538),
 ('Bx10', 534),
 ('B45', 534),
 ('Bx26', 533),
 ('B103', 531),
 ('Q64', 529),
 ('Bx22', 525),
 ('M72', 523),
 ('B41', 520),
 ('M5', 520),
 ('B38', 520),
 ('Q35', 519),
 ('M66', 518),
 ('B62', 513),
 ('Q88', 508),
 ('Q30', 507),
 ('Q84', 507),
 ('Q20A', 505),
 ('S79-SBS', 505),
 ('Q31', 504),
 ('B11', 503),
 ('M35', 496),
 ('Q17', 495),
 ('M10', 495),
 ('Q28', 492),
 ('Q13', 492),
 ('S57', 490),
 ('M31', 490),
 ('B9', 489),
 ('Bx28', 489),
 ('Q76', 488),
 ('S74', 486),
 ('S76', 486),
 ('S78', 486),
 ('S55', 486),
 ('M21', 485),
 ('M57', 485),
 ('B8', 485),
 ('Q85', 485),
 ('M7', 483),
 ('Q23', 482),
 ('Q27', 481),
 ('Q20B', 481),
 ('Q60', 479),
 ('Q4', 479),
 ('M104', 477),
 ('M11', 477),
 ('S56', 476),
 ('B70', 476),
 ('M14A', 475),
 ('B100', 475),
 ('B31', 475),
 ('B2', 475),
 

In [12]:
# Subway
def extractSchool_subway(partitionId, rows):
    if partitionId == 0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) == 58 and fields[11] != 'N/A':
            for i in range(len(fields[11].split('; '))): # split column named subway by ';'
                # further split by 'to' and only take the part before 'to'
                # then split the part before 'to' by ','
                for j in range(len(fields[11].split('; ')[i].split(' to ')[0].split(', '))):
                    yield(fields[0], fields[11].split('; ')[i].split(' to ')[0].split(', ')[j])
                    
school_subway = schools.mapPartitionsWithIndex(extractSchool_subway)
school_subway.take(5)

[('01M292', 'B'),
 ('01M292', 'D'),
 ('01M292', 'F'),
 ('01M292', 'J'),
 ('01M292', 'M')]

In [13]:
school_subway.join(satScores).values() \
    .mapValues(lambda x: (x[0]*x[1], x[1])) \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: int(x[0]/x[1])) \
    .sortBy(lambda x: x[1], ascending=False) \
    .take(10)
# print the subway lines with top 10 SAT scores

[('3', 513),
 ('A', 510),
 ('C', 510),
 ('R', 508),
 ('G', 503),
 ('D', 502),
 ('E', 501),
 ('1', 499),
 ('SIR', 498),
 ('4', 495)]